In  a docker terminal run
jupyter notebook --ip=0.0.0.0 --port=8888 --no-browser --allow-root

then open jupyter in the printed link (such as http://127.0.0.1:8888/tree?token=4d9d578397aede50fc5bd2d92794b562a6bbcbc73b2dfe51)

CODE HERE, REFRESH AND EXECUTE IN THE BROWSER

In [20]:
import os
import sys
import django

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


In [21]:
sys.path.append('/app')

os.environ['DJANGO_SETTINGS_MODULE'] = 'app.settings'
os.environ['PYTHONPATH'] = '/app'

django.setup()

In [22]:
from toolbox import models
from django.forms.models import model_to_dict

In [50]:
project= {
    "id":None,
    "name":"",
    "updated":None,
    "description":"",
    "userField":56,
    "toolboxType":1,
    "infiltration":{
        "id":None,
        "sink_area_min":None,
        "sink_area_max":None,
        "sink_volume_min":None, 
        "sink_volume_max":None,
        "sink_depth_min":None,
        "sink_depth_max":None,
        "sink_index_soil_min":None,
        "sink_index_soil_max":None,
        "sink_land_use":["farmland","forest_conifers","forest_conifers_and_deciduous_trees","grassland","woody_area"],
        "sink_selected":[125, 139, 76288,75305,75114,78200,78662,76222,75372,76163,76790,76266],
        "enlarged_sink_area_min":None,
        "enlarged_sink_area_max":None,
        "enlarged_sink_volume_min":None,
        "enlarged_sink_volume_max":None,
        "enlarged_sink_depth_min":None,
        "enlarged_sink_depth_max":None,
        "enlarged_sink_volume_construction_barrier_min":None,
        "enlarged_sink_volume_construction_barrier_max":None,
        "enlarged_sink_volume_gained_min":None,
        "enlarged_sink_volume_gained_max":None,
        "enlarged_sink_index_soil_min":None,
        "enlarged_sink_index_soil_max":None,
        "enlarged_sink_land_use":["farmland","forest_conifers","forest_conifers_and_deciduous_trees","grassland","woody_area"],
        "enlarged_sink_selected":[],
        "stream_min_surplus_volume_min":None,
        "stream_min_surplus_volume_max":None,
        "stream_mean_surplus_volume_min":None,
        "stream_mean_surplus_volume_max":None,
        "stream_max_surplus_volume_min":None,
        "stream_max_surplus_volume_max":None,
        "stream_plus_days_min":None,
        "stream_plus_days_max":None,
        "stream_distance_to_userfield":0,
        "lake_min_surplus_volume_min":None,
        "lake_min_surplus_volume_max":None,
        "lake_mean_surplus_volume_min":None,
        "lake_mean_surplus_volume_max":None,
        "lake_max_surplus_volume_min":None,
        "lake_max_surplus_volume_max":None,
        "lake_plus_days_min":None,
        "lake_plus_days_max":None,
        "lake_distance_to_userfield":0,
        "weighting_overall_usability":20,
        "weighting_soil_index":80,
        "weighting_forest_field_capacity":33,
        "weighting_forest_hydraulic_conductivity_1m":33,
        "weighting_forest_hydraulic_conductivity_2m":33,
        "weighting_agriculture_field_capacity":33,
        "weighting_agriculture_hydromorphy":33,
        "weighting_agriculture_soil_type":33,
        "weighting_grassland_field_capacity":25,
        "weighting_grassland_hydromorphy":25,
        "weighting_grassland_soil_type":25,
        "weighting_grassland_soil_water_ratio":25
        }}

In [52]:
sink_ids = toolbox_project['infiltration'].get('sink_selected')
sinks = models.Sink4326.objects.filter(id__in=sink_ids)

In [25]:
s_dicts = []
for s in sinks:
        #
        s_dicts.append(s)

In [27]:
one = sinks[4]

In [36]:
ssps = models.SinkSoilProperties.objects.filter(sink__in=sinks)

In [37]:
len(ssps)

10

In [69]:
weighting_overall_usability=int(project['infiltration'].get('weighting_overall_usability', 20))/100
weighting_soil_index=int(project['infiltration'].get('weighting_soil_index', 80))/100
weighting_forest_field_capacity = int(project['infiltration'].get('weighting_forest_field_capacity', 33.3))/100
weighting_forest_hydraulic_conductivity_1m= int(project['infiltration'].get('weighting_forest_hydraulic_conductivity_1m', 33.3))/100
weighting_forest_hydraulic_conductivity_2m= int(project['infiltration'].get('weighting_forest_hydraulic_conductivity_2m', 33.3))/100
weighting_agriculture_field_capacity = int(project['infiltration'].get('weighting_agriculture_field_capacity', 33.3))/100
weighting_agriculture_hydromorphy = int(project['infiltration'].get('weighting_agriculture_hydromorphy', 33.3))/100
weighting_agriculture_soil_type= int(project['infiltration'].get('weighting_agriculture_soil_type', 33.3))/100
weighting_grassland_field_capacity = int(project['infiltration'].get('weighting_grassland_field_capacity', 25))/100
weighting_grassland_hydromorphy = int(project['infiltration'].get('weighting_grassland_hydromorphy', 25))/100
weighting_grassland_soil_type = int(project['infiltration'].get('weighting_grassland_soil_type', 25))/100
weighting_grassland_soil_water_ratio=int(project['infiltration'].get('weighting_grassland_soil_water_ratio', 25))/100

for sink in sinks:
    sps = models.SinkSoilProperties.objects.filter(sink=sink)
    print(len(sps))
    if len(sps) > 0:
        index_be_total = 0
        for sp in sps:
            print(sp)
            index_be = 0
            index_1 = 0
            index_2 = 0
            
            props = sp.soil_properties
            bool_general = (not props.nitrate_contamination) and (not props.waterlog)
            index_1 = bool_general * props.groundwater_distance.rating_index
            print('index_1', index_1)
            
            
            if props.agricultural_landuse.name == 'grassland': # id=1
                index_2 = \
                    weighting_grassland_field_capacity * props.fieldcapacity.rating_index + \
                    weighting_grassland_hydromorphy * props.hydromorphy.rating_index + \
                    weighting_grassland_soil_type * props.soil_texture.rating_index + \
                    weighting_grassland_soil_water_ratio * props.wet_grassland.rating_index

            elif props.agricultural_landuse.name == 'no_agricultural_use': # id=2
                print('weighting_forest_field_capacity', weighting_forest_field_capacity)
                index_2 = \
                    weighting_forest_field_capacity * props.fieldcapacity.rating_index + \
                    weighting_forest_hydraulic_conductivity_1m * props.hydraulic_conductivity_1m_rating + \
                    weighting_forest_hydraulic_conductivity_2m * props.hydraulic_conductivity_2m_rating

            else: #id = 3, fields
                print('weighting_agriculture_hydromorphy', weighting_agriculture_hydromorphy)
                print('props.hydromorphy.rating_index', props.hydromorphy.rating_index)
                index_2 = \
                    weighting_agriculture_field_capacity * props.fieldcapacity.rating_index + \
                    weighting_agriculture_hydromorphy * props.hydromorphy.rating_index + \
                    weighting_agriculture_soil_type * props.soil_texture.rating_index
                    
            print('index_2', index_2)
            index_be = weighting_overall_usability * index_1 + weighting_soil_index * index_2

            index_be_total += index_be * sp.percent_of_total_area

        print('index_be_total', index_be_total)
            
    else:
        print('error')

2
SinkSoilProperties object (885)
index_1 1.0
weighting_agriculture_hydromorphy 0.33
props.hydromorphy.rating_index 1.0
index_2 0.8844000000000001
SinkSoilProperties object (54510)
index_1 1.0
weighting_agriculture_hydromorphy 0.33
props.hydromorphy.rating_index 1.0
index_2 0.9240000000000002
index_be_total 0.9076394543657137
3
SinkSoilProperties object (921)
index_1 1.0
weighting_agriculture_hydromorphy 0.33
props.hydromorphy.rating_index 1.0
index_2 0.8844000000000001
SinkSoilProperties object (873)
index_1 1.0
weighting_agriculture_hydromorphy 0.33
props.hydromorphy.rating_index 1.0
index_2 0.9075
SinkSoilProperties object (84109)
index_1 1.0
weighting_forest_field_capacity 0.33
index_2 0.9240000000000002
index_be_total 0.9088695325383049
1
SinkSoilProperties object (27431)
index_1 0.43
weighting_forest_field_capacity 0.33
index_2 0.9240000000000002
index_be_total 0.8252000000000002
1
SinkSoilProperties object (5790)
index_1 0.29
weighting_forest_field_capacity 0.33
index_2 0.99
ind